# Data Preparation for Multiple Linear Regression

**Purpose:** Merge all datasets to create analysis-ready data for bias detection

**Inputs:**
- population_metrics_demo.csv (suitability scores)
- demographics.csv (ethnicity, sentence length, county)
- current_commits_clean.csv (offense details)
- selection_criteria.xlsx (offense severity tables)

**Output:**
- regression_analysis_data.csv (merged dataset ready for MLR)

## Step 1: Import Libraries

## Configuration: Data Source

Choose where to load data from:
- **GitHub**: Uses Redo.io's public datasets (no local files needed)
- **Local**: Uses files in your `data/` directory

In [ ]:
# Data source configuration
USE_GITHUB = True  # Set to False to use local files

if USE_GITHUB:
    print("Using GitHub data sources")
    BASE_URL = "https://raw.githubusercontent.com/redoio/resentencing_data_initiative/main/data/"
    DATA_PATHS = {
        'demographics': f"{BASE_URL}demographics.csv",
        'current_commits': f"{BASE_URL}current_commitments_clean.csv",
        'pop_metrics': "https://raw.githubusercontent.com/redoio/population_metrics/main/outputs/population_metrics.csv",
        'selection': "https://raw.githubusercontent.com/redoio/offenses_data/main/selection_criteria.xlsx"
    }
else:
    print("Using local data files")
    from pathlib import Path
    data_dir = Path("../data")
    outputs_dir = Path("../outputs")
    DATA_PATHS = {
        'demographics': data_dir / "demographics.csv",
        'current_commits': data_dir / "current_commitments_clean.csv",
        'pop_metrics': outputs_dir / "population_metrics.csv",
        'selection': data_dir / "selection_criteria.xlsx"
    }

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

## Step 2: Set File Paths

In [ ]:
# Define paths
data_dir = Path("../data")
outputs_dir = Path("../outputs")

## Step 3: Load Population Metrics (Suitability Scores)

In [ ]:
# Load population metrics (suitability scores)
pop_metrics = pd.read_csv(DATA_PATHS['pop_metrics'])
print(f"Loaded {len(pop_metrics):,} records")

In [ ]:
# Check shape and columns
pop_metrics.shape

In [ ]:
pop_metrics.head()

## Step 4: Load Demographics (Ethnicity, Sentence Length)

In [ ]:
# Load demographics data
demographics = pd.read_csv(DATA_PATHS['demographics'])
print(f"Loaded {len(demographics):,} records")

In [ ]:
# Select relevant columns
demo_cols = [
    'cdcno',
    'ethnicity',
    'aggregate sentence in months',
    'controlling case sentencing county',
    'offense category',
    'time served in years'
]

demographics_subset = demographics[demo_cols]

In [ ]:
demographics_subset.head()

## Step 5: Load Current Commits (Offense Details)

In [ ]:
# Load current commitments
current_commits = pd.read_csv(DATA_PATHS['current_commits'])
print(f"Loaded {len(current_commits):,} records")

In [ ]:
# Select relevant columns
current_cols = [
    'cdcno',
    'offense_clean',
    'offense description',
    'offense category'
]

current_subset = current_commits[current_cols]

In [ ]:
# Keep only the primary offense (first row per person)
# Assumption: First offense listed is the controlling offense
current_primary = current_subset.groupby('cdcno').first().reset_index()

In [ ]:
current_primary.head()

## Step 6: Load Selection Criteria (Offense Severity Tables)

In [ ]:
# Load offense severity classifications
selection = pd.read_excel(DATA_PATHS['selection'], sheet_name='Penal codes')
print(f"Loaded {len(selection):,} offense codes")

In [ ]:
# Clean offense codes in selection criteria (ensure matching format)
selection['Offenses'] = selection['Offenses'].astype(str).str.strip()

In [ ]:
# Create lookup dictionary: offense_code -> table
offense_to_table = dict(zip(selection['Offenses'], selection['Table']))

# Create lookup dictionary: offense_code -> type
offense_to_type = dict(zip(selection['Offenses'], selection['Type']))

In [ ]:
# Preview mappings
list(offense_to_table.items())[:10]

## Step 7: Map Offense Codes to Severity Tables

In [ ]:
# Add offense table classification to current commits
current_primary['offense_table'] = current_primary['offense_clean'].map(offense_to_table)
current_primary['offense_type'] = current_primary['offense_clean'].map(offense_to_type)

In [ ]:
# Fill missing with "Other" (offenses not in any table)
current_primary['offense_table'].fillna('Other', inplace=True)
current_primary['offense_type'].fillna('Other', inplace=True)

In [ ]:
# Check distribution of offense tables
current_primary['offense_table'].value_counts()

## Step 8: Merge All Datasets

In [ ]:
# Merge population metrics with demographics
analysis_data = pop_metrics.merge(
    demographics_subset,
    on='cdcno',
    how='inner'
)

In [ ]:
# Merge with current commits (offense details)
analysis_data = analysis_data.merge(
    current_primary,
    on='cdcno',
    how='left',
    suffixes=('_demo', '_offense')
)

In [ ]:
# Check merge results
analysis_data.shape

In [ ]:
analysis_data.head()

## Step 9: Data Cleaning and Filtering

In [ ]:
# Convert sentence to numeric (handle any errors)
analysis_data['aggregate sentence in months'] = pd.to_numeric(
    analysis_data['aggregate sentence in months'],
    errors='coerce'
)

In [ ]:
# Drop rows with missing critical variables
analysis_data_clean = analysis_data.dropna(subset=[
    'aggregate sentence in months',
    'ethnicity',
    'score'
])

In [ ]:
# Remove outliers (sentences > 50 years = 600 months)
# These are likely life sentences or data errors
analysis_data_clean = analysis_data_clean[
    (analysis_data_clean['aggregate sentence in months'] > 0) &
    (analysis_data_clean['aggregate sentence in months'] <= 600)
]

In [ ]:
# Check final sample size
print(f"Original sample: {len(analysis_data):,}")
print(f"After cleaning: {len(analysis_data_clean):,}")
print(f"Dropped: {len(analysis_data) - len(analysis_data_clean):,}")

## Step 10: Create Additional Variables

In [ ]:
# Create binary variable: High suitability (score above median)
median_score = analysis_data_clean['score'].median()
analysis_data_clean['high_suitability'] = (analysis_data_clean['score'] > median_score).astype(int)

In [ ]:
# Create categorical variable: Suitability tertiles (low/medium/high)
analysis_data_clean['suitability_category'] = pd.cut(
    analysis_data_clean['score'],
    bins=[0, 1, 2, 3],
    labels=['Low', 'Medium', 'High'],
    include_lowest=True
)

In [ ]:
# Create binary variable: Violent offense (Table B or C)
analysis_data_clean['violent_offense'] = analysis_data_clean['offense_table'].isin(['Table B', 'Table C']).astype(int)

## Step 11: Descriptive Statistics

In [ ]:
# Summary statistics for continuous variables
analysis_data_clean[[
    'aggregate sentence in months',
    'score',
    'desc_nonvio_curr',
    'desc_nonvio_past',
    'severity_trend'
]].describe()

In [ ]:
# Frequency tables for categorical variables
print("Ethnicity Distribution:")
print(analysis_data_clean['ethnicity'].value_counts())
print()

print("Offense Table Distribution:")
print(analysis_data_clean['offense_table'].value_counts())
print()

print("Top 10 Counties:")
print(analysis_data_clean['controlling case sentencing county'].value_counts().head(10))

## Step 12: Check for Racial Disparities (Preliminary)

In [ ]:
# Mean sentence by ethnicity
analysis_data_clean.groupby('ethnicity')['aggregate sentence in months'].agg(['mean', 'median', 'count'])

In [ ]:
# Mean suitability score by ethnicity
analysis_data_clean.groupby('ethnicity')['score'].agg(['mean', 'median'])

## Step 13: Save Analysis-Ready Dataset

In [ ]:
# Save to CSV
output_path = outputs_dir / "regression_analysis_data.csv"
analysis_data_clean.to_csv(output_path, index=False)

In [ ]:
print(f"Saved: {output_path}")
print(f"Rows: {len(analysis_data_clean):,}")
print(f"Columns: {len(analysis_data_clean.columns)}")

## Step 14: Export Column List for Reference

In [ ]:
# List all columns in final dataset
print("Columns in regression_analysis_data.csv:")
for col in analysis_data_clean.columns:
    print(f"  - {col}")

## Summary

**Dataset Created:** `regression_analysis_data.csv`

**Key Variables:**
- **Outcome:** `aggregate sentence in months`
- **Main Predictor:** `ethnicity`
- **Controls:** `score`, `offense_table`, `controlling case sentencing county`
- **Individual Metrics:** `desc_nonvio_curr`, `desc_nonvio_past`, `severity_trend`

**Ready for:**
- Multiple Linear Regression (Model 1, 2, 3)
- Interaction analysis